# TASK4. 추가적인 탐색적 데이터 분석(EDA)

다양한 모델을 활용해 EDA를 분석하는 과정입니다. 다음 예제는 분석 참고용입니다.

## 0. 환경변수 설정

In [ ]:
from google.cloud import bigquery
import os

client = bigquery.Client()

PROJECT_ID = client.project
DATASET_NAME = "cymbal"
TABLE_NAME = "customer_review"
BUCKET_NAME = "qwiklabs-gcp-01-416cbdc92d8c-bucket" #qwiklabs-gcp-01-416cbdc92d8c-bucket
TASK4_1_NEW_TABLE_NAME = "embeded_table"
TASK4_2_NEW_TABLE_NAME = "cgenf_table" #category enforcement table
TASK4_3_NEW_TABLE_NAME = "improv_table" #improvement enforcement table
TASK4_1_MODEL_NAME = "embedding_model"
TASK4_2_MODEL_NAME = "gemini_model"
TASK4_3_MODEL_NAME = "gemini_model"
TASK4_5_MODEL_NAME = "gemini_model"


## 1. 텍스트 임베딩 모델 생성 및 유사도 검색

In [ ]:
task4_1_1_query = f"""
 CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_NAME}.{TASK4_1_NEW_TABLE_NAME}` AS
 SELECT * FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_1_MODEL_NAME}`,
  (SELECT review_text AS content, customer_id, customer_review_id
   FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
  )
)
"""


In [ ]:
%%bigquery --project $PROJECT_ID
$task4_1_1_query

Query is running:   0%|          |

""


In [ ]:
task4_1_2_query = f"""
  SELECT
    t.content,
    t.customer_id,
    t.customer_review_id,
    # 코사인 유사도 점수 (1에 가까울수록 의미적으로 유사함)
    1 - ML.DISTANCE(
      t.ml_generate_embedding_result,
      query_embed.ml_generate_embedding_result,
      'COSINE'
    ) AS similarity_score
  FROM
    `bq-agent-kr.cymbal.embeded_table` AS t,
    ML.GENERATE_EMBEDDING(
    MODEL `bq-agent-kr.cymbal.embedding_model`,
    (SELECT '품질 불량' AS content) # 여기에 검색어를 입력합니다.
    ) AS query_embed
  ORDER BY similarity_score DESC
  LIMIT 5
"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_1_2_query

Query is running:   0%|          |

Downloading:   0%|          |

,content,customer_id,customer_review_id,similarity_score
0,The product quality is very poor. I definitely...,8857,249,0.764613
1,The product quality is very poor. I definitely...,8858,203,0.764613
2,The product quality is very poor. I definitely...,10008,209,0.764613
3,The product quality is very poor. I definitely...,4533,242,0.764613
4,The product quality is very poor. I definitely...,9823,201,0.764613


## 2. 각 리뷰별 카테고리 군집화 데이터 보강하기

어떻게 데이터가 보강되는지 코드와 예시를 보고 이해합니다.

In [ ]:
task4_2_1_query = f"""
  SELECT
    customer_review_id,
    customer_id,
    issue_category
  FROM
    AI.GENERATE_TABLE(
      MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_2_MODEL_NAME}`,
      (
        SELECT
          t.customer_id,
          t.customer_review_id,
          t.review_text,
          CONCAT(
            '''
            부정적인 리뷰로 판단된 경우 분석해서 문제점을 카테고리화해서 분류하려고 해
  .           다음 카테고리 중 하나를 택해서 분류해줘
            **품질불량, 배송문제, 기대미충족, 단순변심, 결제문제, 응대문제, 기타**
              긍정적인 리뷰로 판단된 경우 **해당없음**으로 분류해주면 돼.
              분석할 리뷰:
            ''',
            t.review_text
          ) AS prompt
        FROM
          `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}` AS t
        WHERE
          t.rating < 3
      ),
      STRUCT(
        "issue_category STRING" AS output_schema,
        8192 AS max_output_tokens
      )
    )

"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_2_1_query

Query is running:   0%|          |

Downloading:   0%|          |

,customer_review_id,customer_id,issue_category
0,221,5516,품질불량
1,202,10959,배송문제
2,227,10387,기대미충족
3,235,6540,배송문제
4,206,5330,품질불량
5,241,3407,응대문제
6,245,5327,기대미충족
7,233,5919,기대미충족
8,207,5424,품질불량
9,210,10877,기대미충족


해당 결과를 아래 과정을 통해 테이블로 생성합니다.

In [ ]:
task4_2_2_query = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_NAME}.{TASK4_2_NEW_TABLE_NAME}` AS (
  SELECT
    customer_review_id,
    customer_id,
    issue_category
  FROM
    AI.GENERATE_TABLE(
      MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_2_MODEL_NAME}`,
      (
        SELECT
          t.customer_id,
          t.customer_review_id,
          t.review_text,
          CONCAT(
            '''
            부정적인 리뷰로 판단된 경우 분석해서 문제점을 카테고리화해서 분류하려고 해
  .           다음 카테고리 중 하나를 택해서 분류해줘
            **품질불량, 배송문제, 기대미충족, 단순변심, 결제문제, 응대문제, 기타**
              긍정적인 리뷰로 판단된 경우 **해당없음**으로 분류해주면 돼.
              분석할 리뷰:
            ''',
            t.review_text
          ) AS prompt
        FROM
          `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}` AS t
        WHERE
          t.rating < 3
      ),
      STRUCT(
        "issue_category STRING" AS output_schema,
        8192 AS max_output_tokens
      )
    )

)

"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_2_2_query

Query is running:   0%|          |

""


## 3. 고객들의 리뷰를 분석해 개선점을 정의한 데이터 보강하기

해당 단계에서는 위의 코드와 다른 함수로 데이터 보강을 하는 방법을 보고 이해합니다.

In [ ]:
task4_3_1_query = f"""
  SELECT
    generated_output.customer_id,
    generated_output.customer_review_id,
    generated_output.ml_generate_text_result.candidates[0].content.parts[0].text AS improvement_points
  FROM
  ML.GENERATE_TEXT(
    MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_3_MODEL_NAME}`,
    (SELECT
  customer_id,
  customer_review_id,
  CONCAT(
    '''
    다음 리뷰를 참고해서 고객이 불만족한 부분을 개선할 수 있는 구체적인 방안 1개와 고객이 좋아할 법한 상품 특성을 1개 제시해줘 양식은 다음과 같아. 양식을 무조건 지켜서 작성해줘.
    개선방안 :
    고객이 좋아할 법한 특성 :
  참고할 리뷰내용 :
  ''', review_text) AS prompt
  FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
  WHERE rating < 3),
  STRUCT(
    0.2 AS temperature,
    2048 AS max_output_tokens )
  ) AS generated_output
"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_3_1_query

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_review_id,improvement_points
0,5516,221,"""개선방안 : 제품의 핵심 부품에 더 견고한 소재를 사용하고, 생산 과정에서 품질 ..."
1,8496,223,"""개선방안 : 제품의 핵심 부품 소재를 고급화하여 내구성을 획기적으로 개선한다.\n..."
2,8887,215,"""개선방안 : 배송 지연 보상 정책 도입 및 실시간 배송 추적 시스템 강화\n고객이..."
3,8686,213,"""개선방안 : 제품의 핵심 기능을 처음 사용하는 고객을 위해 단계별로 따라 할 수 ..."
4,6540,235,"""개선방안 : 제품 포장재를 더 견고한 소재로 변경하고, 포장 과정에 대한 품질 검..."
5,5330,206,"""개선방안 : 생산 공정 전반에 걸쳐 품질 검수 단계를 강화하여 불량률을 낮추고, ..."
6,3407,241,"""개선방안 : 고객 문의 처리 절차를 간소화하고, 각 단계별 처리 시간을 단축하기 ..."
7,5327,245,"""개선방안 : 제품 상세 페이지에 실제 사용 환경을 반영한 사진이나 영상을 추가하여..."
8,5919,233,"""개선방안 : 경쟁사 제품 대비 핵심 기능의 성능을 대폭 강화하여 고객이 체감하는 ..."
9,5424,207,"""개선방안 : 핵심 부품의 재질을 고급화하여 제품의 내구성을 강화하고, 이를 통해 ..."


해당 결과를 아래 과정을 통해서 테이블로 생성합니다.

In [ ]:
task4_3_2_query = f"""
  CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_NAME}.{TASK4_3_NEW_TABLE_NAME}` AS (
    SELECT
      generated_output.customer_id,
      generated_output.customer_review_id,
      generated_output.ml_generate_text_result.candidates[0].content.parts[0].text AS improvement_points
    FROM
    ML.GENERATE_TEXT(
      MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_3_MODEL_NAME}`,
      (SELECT
    customer_id,
    customer_review_id,
    CONCAT(
      '''
      다음 리뷰를 참고해서 고객이 불만족한 부분을 개선할 수 있는 구체적인 방안 1개와 고객이 좋아할 법한 상품 특성을 1개 제시해줘 양식은 다음과 같아. 양식을 무조건 지켜서 작성해줘.
      개선방안 :
      고객이 좋아할 법한 특성 :
    참고할 리뷰내용 :
    ''', review_text) AS prompt
    FROM `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
    WHERE rating < 3),
    STRUCT(
      0.2 AS temperature,
      2048 AS max_output_tokens )
    ) AS generated_output
  )
"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_3_2_query

Query is running:   0%|          |

""


## 4. 분석 및 보강한 데이터 결합하기

In [ ]:
task4_4_1_query = f"""
  SELECT
    t1.*,
    t2.issue_category
  FROM `{PROJECT_ID}.{DATASET_NAME}.{TASK4_2_NEW_TABLE_NAME}` as t2
  JOIN `{PROJECT_ID}.{DATASET_NAME}.{TASK4_3_NEW_TABLE_NAME}` as t1
  ON t1.customer_review_id = t2.customer_review_id
"""

In [ ]:
%%bigquery --project $PROJECT_ID
$task4_4_1_query

Query is running:   0%|          |

Downloading:   0%|          |

,customer_id,customer_review_id,improvement_points,issue_category
0,6418,212,"""개선방안 : 제품 생산 과정에 다단계 품질 관리(QC) 시스템을 도입하여 불량률을...",품질불량
1,5919,233,"""개선방안 : 경쟁 제품 분석을 통해 고객이 중요하게 생각하는 핵심 기능이나 성능을...",기대미충족
2,3407,241,"""개선방안 : 고객 문의 접수 후 24시간 이내 1차 답변 및 72시간 이내 문제 ...",응대문제
3,4264,205,"""개선방안 : 사용법을 직관적으로 이해할 수 있도록 제품 내부에 인터랙티브 튜토리얼...",기대미충족
4,10769,222,"""개선방안 : 제품 사진 촬영 시 실제 색상과 디자인을 가장 정확하게 반영하도록 전...",기대미충족
5,2057,218,"""개선방안 : 앱 내 오류 보고 시스템을 강화하여, 기능 오작동 발생 시 사용자가 ...",품질불량
6,6355,219,"""개선방안 : 기능 오류 발생 시 즉각적인 원인 분석 및 패치 업데이트 시스템을 구...",품질불량
7,6540,235,"""개선방안 : 모든 제품에 대해 '안심 포장 가이드라인'을 수립하여 완충재 사용을 ...",배송문제
8,5744,240,"""개선방안 : 제품의 핵심 부품에 대해 더 강성이 높은 소재를 적용하고, 출고 전 ...",품질불량
9,6531,243,"""개선방안 : 핵심 기능에 대한 자동화된 기능 테스트 및 사용자 인수 테스트(UAT...",품질불량


## 5. 제품 테이블에 제품의 장점, 단점 그리고 특성을 정의한 데이터 추가하기

In [ ]:
task4_5_1_query = f"""
  SELECT
    generated_output.id,
    generated_output.title,
    generated_output.uri,
    generated_output.availability,
    generated_output.categories,
    generated_output.product_id,
    generated_output.price,
    generated_output.average_rating,
    generated_output.review_count,
    generated_output.ml_generate_text_result.candidates[0].content.parts[0].text AS products_details
  FROM
    ML.GENERATE_TEXT(
      MODEL `{PROJECT_ID}.{DATASET_NAME}.{TASK4_5_MODEL_NAME}`,
      (
        SELECT
          *,
          CONCAT(
            '''
            다음 uri를 참고해서 제품의 장점과 단점 그리고 특성을 1개씩 적어줘.
            다음 양식을 무조건 지켜줘
            장점 :
            단점 :
            특성 :
            참고할 uri :
            ''',
            uri
          ) AS prompt
        FROM
          `{PROJECT_ID}.{DATASET_NAME}.products`
      ),
      STRUCT(
        0.2 AS temperature,
        2048 AS max_output_tokens
      )
    ) AS generated_output
"""


In [ ]:
%%bigquery --project $PROJECT_ID
$task4_5_1_query

Query is running:   0%|          |

Downloading:   0%|          |

,id,title,uri,availability,categories,product_id,price,average_rating,review_count,products_details
0,GGOEGBJC100199,Google Mesh Bag Blue,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Bags],23382,7800,3.0,717,"""장점 : 가볍고 통기성이 좋아 해변, 헬스장 등 다양한 용도로 활용하기 좋다.\n..."
1,GGOEGBRC127999,Google Incognito Techpack V2,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Bags],34069,114400,3.6,610,"""장점 : 재활용 소재를 사용하여 환경 친화적입니다.\n단점 : 120달러로 가격이..."
2,GGOEGBRJ114199,Google Utility BackPack,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Bags],45465,156000,3.5,641,"""장점 : 다양한 수납공간과 편안한 착용감을 제공합니다.\n단점 : 부분 세척만 가..."
3,GGCOGOCC101199,#IamRemarkable Journal,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Office],31736,10400,4.0,847,"""장점 : 자신의 성과를 기록하고 목표를 설정하며 자신감을 높이는 데 도움을 줍니다..."
4,GGOEAOAL129199,Android Iconic Pen,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Office],46816,2275,3.5,716,"""장점 : 부드러운 필기감\n단점 : 일반 펜에 비해 높은 가격\n특성 : 아이코닉..."
...,...,...,...,...,...,...,...,...,...,...
1263,GGOEGAET090513,Google Tee Yellow,https://shop.googlemerchandisestore.com/Google...,OUT_OF_STOCK,[Apparel],21553,28600,4.1,893,"""장점 : 구글 브랜드에 대한 소속감이나 자부심을 표현할 수 있다.\n단점 : 디자..."
1264,GGOEGAET090518,Google Tee Yellow,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Apparel],41961,28600,3.8,365,"""장점 : 편안한 착용감 (100% 빗질한 링 스펀 면으로 부드럽고 편안함)\n단점..."
1265,GGOEGAET090512,Google Tee Yellow,https://shop.googlemerchandisestore.com/Google...,IN_STOCK,[Apparel],49471,28600,4.7,60,"""장점 : 100% 면 소재로 편안한 착용감\n단점 : 로고 부분 다림질 불가 (장..."
1266,GGOEGAET090515,Google Tee Yellow,https://shop.googlemerchandisestore.com/Google...,OUT_OF_STOCK,[Apparel],31145,28600,4.6,481,"""장점 : 부드럽고 편안한 착용감\n단점 : 기본 티셔츠치고는 다소 높은 가격\n특..."


## 6. 분석결과 CSV로 저장하기

해당 단계에서는 보강한 데이터를 CSV 타입으로 버킷에 저장하는 과정입니다.

In [ ]:
customer_review_enforcement = f"{PROJECT_ID}.{DATASET_NAME}.customer_review_enf_result"

job_config = bigquery.QueryJobConfig(
    destination=customer_review_enforcement,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # 테이블이 있으면 덮어쓰기
)

query_job = client.query(task4_4_1_query, job_config=job_config)
query_job.result()

table_ref = client.dataset(DATASET_NAME).table("customer_review_enf_result")

destination_uri = f"gs://{BUCKET_NAME}/task4_result/customer_review_enf_result.csv"

extract_job_config = bigquery.ExtractJobConfig(
    destination_format=bigquery.DestinationFormat.CSV,
    print_header=True,  # 첫 줄에 헤더(컬럼 이름) 포함
    field_delimiter=',' # 구분자 쉼표(CSV) 사용
)

# 테이블 추출 작업 시작
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    job_config=extract_job_config
)

# 작업 완료 대기
extract_job.result()

print(f"테이블 '{table_ref.path}'의 데이터가 GCS '{destination_uri}'에 CSV 파일로 저장되었습니다.")

테이블 '/projects/bq-agent-kr/datasets/cymbal/tables/customer_review_enf_result'의 데이터가 GCS 'gs://school_anlayze/task4_result/customer_review_enf_result.csv'에 CSV 파일로 저장되었습니다.


In [ ]:
product_enforcement = f"{PROJECT_ID}.{DATASET_NAME}.product_enf_result"

job_config = bigquery.QueryJobConfig(
    destination=product_enforcement,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # 테이블이 있으면 덮어쓰기
)

query_job = client.query(task4_5_1_query, job_config=job_config)
query_job.result()

table_ref = client.dataset(DATASET_NAME).table("product_enf_result")

destination_uri = f"gs://{BUCKET_NAME}/task4_result/product_enf_result.csv"

extract_job_config = bigquery.ExtractJobConfig(
    destination_format=bigquery.DestinationFormat.CSV,
    print_header=True,  # 첫 줄에 헤더(컬럼 이름) 포함
    field_delimiter=',' # 구분자 쉼표(CSV) 사용
)

# 테이블 추출 작업 시작
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    job_config=extract_job_config
)

# 작업 완료 대기
extract_job.result()

print(f"테이블 '{table_ref.path}'의 데이터가 GCS '{destination_uri}'에 CSV 파일로 저장되었습니다.")

테이블 '/projects/bq-agent-kr/datasets/cymbal/tables/product_enf_result'의 데이터가 GCS 'gs://school_anlayze/task4_result/product_enf_result.csv'에 CSV 파일로 저장되었습니다.
